#  Customizing Nodes

The [Hello, FABRIC](./hello_fabric.ipynb) notebook demonstrates how to create a simple node on a random FABRIC site with default characteristics.   Most experiments require more control over the placement and configuration of your nodes.  

This notebook, will show how to set the following properties on a node:

- Site: Choose the FABRIC site where your node will be hosted.
- Host: Choose the specific worker node that the VM will run on.
- Cores: Number of cores, amount of RAN
- RAM: Amount of RAM
- Disk: Amount of local disk space
- VM Image: The operating system image used for the VM


## Setup the Experiment

#### Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slices

Create the slice and set the specific node attributes.  Note that the cores, ram, and disk are only *hints*.  The actual values will be the closest instance type that is larger than the chosen values.

Amounts of cores, ram, and disk will be rounded up to the closest instance type. These amounts should be considered minimums rather than specific requirements. You may find [this article](https://learn.fabric-testbed.net/knowledge-base/how-vms-are-sized-in-fabric/) useful in explaining how VM size allocation works in FABRIC.

Note that your project must have `VM.NoLimit` permission set in order to allocate VMs larger than 2 cores, 10G of RAM and 10G of disk. Otherwise you will receive an error of the type `PDP Authorization check failed - Policy Violation: Your project is lacking VM.NoLimitCPU or VM.NoLimit tag to provision VM with more than 2 cores.`

In [ ]:
slice_name = 'MySlice1'

#Create Slice
slice = fablib.new_slice(slice_name)

# Add node
node = slice.add_node(name='Node1', 
                      site='MAX',
                      cores=4, 
                      ram=16, 
                      disk=100, 
                      image='default_ubuntu_20')

#Submit Slice Request
slice.submit()

Alternatively, you can specify a specific instance type by name. A full list of available instance types is [here](https://github.com/fabric-testbed/InformationModel/blob/master/fim/slivers/data/instance_sizes.json).

In [ ]:
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

slice_name = 'MySlice2'

#Create Slice
slice = fablib.new_slice(slice_name)

# Add node
node = slice.add_node(name='Node1', 
                      site='MAX',
                      instance_type='fabric.c8.m32.d100',
                      image='default_ubuntu_20')

#Submit Slice Request
slice.submit()


In some cases you may need to allocate your VM on a specific worker node in a given site. All worker nodes are named `<site name>-w<worker index>.fabric-testbed.net`. 

Note however that FABRIC workers have different assortments of devices and certain combinations of PCI devices/components may not be possible on a given worker node in which case your slice may fail. In general this feature should not be needed often. 

In [ ]:
slice_name = 'MySlice3'

#Create Slice
slice = fablib.new_slice(slice_name)

# Add node
node = slice.add_node(name='Node1', 
                      site='MAX',
                      # use the host parameter to force a specific worker
                      host='max-w2.fabric-testbed.net',
                      cores=4, 
                      ram=16, 
                      disk=100, 
                      image='default_ubuntu_20')

#Submit Slice Request
slice.submit()

Note that the 'Host' column in the Nodes table now shows `max-w2.fabric-testbed.net` indicating the VM is on the requested worker node.

## Delete the slices

In [ ]:
for slice_index in range(1,4):
    print(f'Deleting slice MySlice{slice_index}')
    try:
        fablib.delete_slice("MySlice" + str(slice_index))
    except:
        pass
